# 0. Initialization Setting

- 문제 개요: 카드 기록을 활용한 사기 거래 탐지
- binary classification 문제로, 온라인 거래가 사기일 확률을 예측하는 것이 목적임
- 대부분 column이 비식별화(Marsking)되어 있음
- 결측치가 많음
- 클래스가 불균형함 (class imbalance 문제)

## Library Setting

In [ ]:
import pandas as pd
import numpy as np

# Visualization Library
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline
plt.rcParams['font.size'] = 15
plt.rcParams['figure.figsize'] = (12, 8)
plt.style.use('ggplot')
color_pal = [x['color'] for x in plt.rcParams['axes.prop_cycle']]
#
from sklearn import preprocessing
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score

#ML modeling method
import xgboost as xgb
from xgboost import XGBClassifier
import lightgbm as lgb
from lightgbm import LGBMClassifier
import catboost as catb
from catboost import CatBoostClassifier

import warnings
warnings.filterwarnings("ignore")

import gc
gc.enable()

import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [ ]:
# Standard plotly imports
# %pip install chart_studio
# import chart_studio.plotly as py
# import plotly.graph_objs as go
# import plotly.tools as tls
# from plotly.offline import iplot, init_notebook_mode
# import cufflinks
# import cufflinks as cf
# import plotly.figure_factory as ff

# #Using plotly + cufflinks in offline mode
# init_notebook_mode(connected=True)
# cufflinks.go_offline(connected=True)


## Hyperparameter Setting

In [ ]:
#Loading input data Part
is_index_TransactionID = False
#========================================================
#EDA Part

#========================================================
#Featrue Engineering Part

#Label Encoding 
#one-hot-encoding/ labelencoderlibrary/ label-encoding
label_encoding_option = 'LabelEncoderLibrary'
#========================================================
#Modeling Part
#1) Dataset Spliting Method for Validation (Just split validation data vs Cross Validation)

#2) Oversmpling of Target data

##2-1) Oversmpling Method (SMOTE vs Borderline SMOTE vs Adaptive Synthetic Sampling(ADASYN))

#3 Modeling Method (Random Forest vs XGBoost vs LightGBM vs CatBoost)

#3-1) Using Ensemble of Ensemble Method (Bagging vs RandomForest vs or not)


# 1. Load Data

## Load Data

In [ ]:
data_path = "../input/ieee-fraud-detection/"

def load_data (data_path, is_index_TransactionID):
    if is_index_TransactionID:
        train_identity = pd.read_csv(data_path + "train_identity.csv",index_col='TransactionID')
        test_identity = pd.read_csv(data_path + "test_identity.csv",index_col='TransactionID')
        train_transaction = pd.read_csv(data_path + "train_transaction.csv",index_col='TransactionID')
        test_transaction = pd.read_csv(data_path + "test_transaction.csv",index_col='TransactionID')
    else:
        train_identity = pd.read_csv(data_path + "train_identity.csv")
        test_identity = pd.read_csv(data_path + "test_identity.csv")
        train_transaction = pd.read_csv(data_path + "train_transaction.csv")
        test_transaction = pd.read_csv(data_path + "test_transaction.csv")
        
    return train_identity, test_identity, train_transaction, test_transaction

In [ ]:
train_identity, test_identity, train_transaction, test_transaction = load_data(data_path, is_index_TransactionID)

### EDA 하기전에 info()함수와 head()함수를 통해 데이터에 대해 간단히 살펴본다

#### train_transaction Dataset

In [ ]:
train_transaction.info()
train_transaction.head(3)

- 590540 개의 data와 394개의 feature가 있음(설명변수 + 반응변수(target))
- Data type이 object(str)인 feature가 14개 있음(Categorical Variable, 범주형 변수가 총 14개) -> label encoding 필요함
- 데이터를 1.7GB이상 사용함 -> 각 feature의 범위에 맞게 데이터 타입을 바꿔 데이터 사용량을 낮출 필요가 있음
- 많은 feature들이 식별이 불가능하고, 결측치(NaN)값이 많이 보임

#### test_transaction Dataset

In [ ]:
test_transaction.info()
test_transaction.head(3)

- 506691 개의 data와 393개의 feature가 있음(Target feature 없음)
- Data type이 object(str)인 feature가 train_transaction과 동일하게 14개 있음 -> label encoding 필요함
- 데이터를 1.5GB이상 사용함 -> 각 feature의 범위에 맞게 데이터 타입을 바꿔 데이터 사용량을 줄일 필요가 있음
- 많은 feature들이 식별이 불가능하고, 결측치(NaN)값이 많이 보임

#### train_identity Dataset

In [ ]:
train_identity.info()
train_identity.head(3)

- 144233 개의 data와 41개의 feature가 있음
- Data type이 object(str)인 17개 있음 -> label encoding 필요함
- 데이터를 약 45.1MB 사용함
- 많은 feature들이 식별이 불가능하고, info함수의 Non-Null Count를 보면 결측치(NaN)값이 데이터 개수와 많이 차이나는 column들이 보임

#### test_identity Dataset

In [ ]:
test_identity.info()
test_identity.head(3)

- 141907 개의 data와 41개의 feature가 있음
- Data type이 object(str)인 17개 있음 -> label encoding 필요함
- 데이터를 약 44.4MB 사용함
- 많은 feature들이 식별이 불가능하고, info함수의 Non-Null Count를 보면 결측치(NaN)값이 데이터 개수와 많이 차이나는 column들이 보임
- id column의 id와 숫자사이의 연결문자가 train_identity가 "_" 인 것과 달리, test_identity는 "-"임

## Reduce Memory Usage

- transaction의 데이터의 크기가 커서 메모리 사용량이 크므로 각 데이터에서 사용하는 숫자의 범위에 맞게 줄일 필요가 있음
- 따라서 모든 Dataset에 대해서 각 데이터에서 사용하는 숫자의 범위에 맞게 Data type을 변경해주는 과정을 거침

In [ ]:
## Memory Reducer Function
# :df pandas dataframe to reduce size   # type: pd.DataFrame()
def reduce_memory_usage(df):
    # 숫자 데이터 형 리스트
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    # 처음 메모리 사용량
    start_memory = df.memory_usage().sum() / 1024**2
    
    for col in df.columns:
        col_type = df[col].dtypes
        # feature(column)의 데이터 형이 numerics안에 있으면
        if col_type in numerics:
            #해당 feature의 최소값, 최대값 찾기 
            c_min = df[col].min()
            c_max = df[col].max()
            # int 형인 경우
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            # float형인 경우
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
                    
    #줄인 메모리 사용량
    end_memory = df.memory_usage().sum() / 1024**2
    print('Memory usage decreased from {:5.2f}MB to {:5.2f}MB ({:.1f}% reduction)'.format(start_memory,end_memory, ((start_memory-end_memory)/start_memory)*100))

In [ ]:
for df in [train_identity, test_identity, train_transaction, test_transaction]:
    reduce_memory_usage(df)

# 2. EDA
- 데이터 전반에 대한 탐색
- Target feature의 class의 분포(비율) 확인(Class Imbalance 문제여부 확인)
- Feature별 연속형 / 범주형 여부, 결측치, Unique값 일부와 개수 확인
- 범주형 변수(Discrete or Categorical Features) 탐색
- 연속형 변수(Continuous or Numerical Features) 탐색

## 2.1 About Data and features

- 'TransactionID', 'TransactionDT'변수는 삭제
- X: df에서 isFraud가 제거된 DF
- Y: df에서 isFraud column만 가져온 Series

train.csv: 모델 학습용 데이터 / test.csv: 모델 평가용 데이터 
 - TransactionID: 거래 ID(비식별화)
 - TransactionDT: 거래 시각(비식별화)
 - TransactionAmt: 거래 금액(단위: US 달러)
 - ProductCD: 상품 또는 제품의 코드
 - addr1 - 2: 주소정보
 - dist: 거리정보
 - card1 - card6: 카드 관련 정보(비식별화)
 - P_emaildomain, R_emaildomain: 이메일 정보
 - C1 - C14: Counting 관련 정보(비식별화)
 - D1 - D15: 거래관련 시간 정보(비식별화)
 - M1 - M9: 기존 거래와의 매칭 정보 (일치 여부정보)
 - V1 - V339: 여러 정보를 포함한 engineered된 정보
 - isFraud: 사기 거래 여부(Target column / Label)
 

Categorical Features
- ProductCD
- card4, card6
- addr1, addr2
- P_emaildomain
- R_emaildomain
- M1 - M9
- DeviceType
- DeviceInfo
- id_12 - id_38
 

### DF.shape 속성을 통해 다시 한 번 각 데이터 셋에 대한 데이터 개수(row수)와 feature 수(column 수) 확인

In [ ]:
print(f'train_transaction shape is {train_transaction.shape}')
print(f'test_transaction shape is {test_transaction.shape}')
print(f'train_identity shape is {train_identity.shape}')
print(f'test_identity shape is {test_identity.shape}')      

- train데이터의 거래데이터 샘플 수가 590540개이고, train데이터의 identity데이터 샘풀 수가 144233개이므로 train data는 144233개의 계정들의 590540번의 거래 데이터라고 이해할 수 있음
- test데이터의 거래데이터 샘플 수가 506691개이고, train데이터의 identity데이터 샘풀 수가 141907개이므로 train data는 141907개의 계정들의 506691번의 거래 데이터라고 이해할 수 있음

### 거래 시각 정보 feature인 TransactionDT에 따른 train_transaction과 test_transaction 데이터의 거래횟수 분포확인

In [ ]:
train_transaction['TransactionDT'].plot(kind='hist', figsize=(15,5), bins=100, label='Train', title='Train vs. Test TransactionDT distribution')
test_transaction['TransactionDT'].plot(kind='hist', label='test', bins=100)
plt.legend()
plt.show()

- 거래 시각 정보 feature인 TransactionDT에 따른 데이터 분포를 확인해본 결과, 
    - train data와 test data가 거래 시각에 대해 데이터들이 서로 섞여 있지 않음
    - 과거의 일정기간동안 이루어진 거래데이터가 train데이터로 설정됨
    - train data의 거래 이후 일정 기간이 지난 뒤, 그 이후의 거래 데이터가 tset data로 설정되어 있음을 확인할 수 있음

- 하지만 train data와 test data가 시간의 흐름에 따라 설정되어 있다고 하더라도 이 정보는 Target을 예측하는데 큰 영향이 없을 것으로 판단됨
    - 해당 문제는 사기거래여부를 예측하는 Binary Classification 문제이므로 어떤 수치값을 예측하는 Regression문제가 아니기 때문임
    - 따라서 각각의 거래 데이터를 독립적으로 봐야함
    - 시간 관련 feature에 대해서는 시간의 영향을 없애도록 전처리가 필요할 것으로 판단됨

## 2.2 Target feature의 class의 분포(비율)를 확인(Class Imbalance 문제여부 확인)

In [ ]:
target = train_transaction['isFraud']
target.value_counts(normalize = True)

In [ ]:
target.value_counts()

In [ ]:
# 종속 변수의 분포 확인(Class imbalance 확인)
from collections import Counter
target_dist_data = Counter(target)
print('Distribution of transaction data by target feature =', target_dist_data)
print('\n')
print('Ratio of fraud transaction data is %.1f%%'%(target_dist_data[0] / len(train_transaction['isFraud'])*100))
print('Ratio of non-fraud transaction data is %.1f%%'%(target_dist_data[1]/ len(train_transaction['isFraud'])*100))

del target, target_dist_data
x = gc.collect()

탐색 내용 정리
- 사기 거래가 아닌 거래가 전체 거래의 약 96.5%를 차지함 
- 본 문제는 예측값의 Class 비율이 서로 차이가 많이 나는(불균형한) class imbalance binary classification problem임

## 2.3 Feature별 연속형 / 범주형 여부, 결측치, Unique값 일부와 개수 확인

In [ ]:
for column in train_transaction.drop('isFraud', axis=1).columns:
    col_Series = train_transaction[column]
    print(f'{column}: DataType: {col_Series.dtype}, 결측치 개수: {col_Series.isnull().sum()}, Unique값 개수: {col_Series.nunique()}, Unique값 일부: {col_Series.unique()[:5]}')
for column in train_identity.columns:
    col_Series = train_identity[column]
    print(f'{column}: DataType: {col_Series.dtype}, 결측치 개수: {col_Series.isnull().sum()}, Unique값 개수: {col_Series.nunique()}, Unique값 일부: {col_Series.unique()[:5]}')
    
del col_Series
x= gc.collect()

탐색 내용 정리
- TransactionAmt는 히스토그램 등을 통해 확인한 결과 연속형 변수가 확실함
- ProductCD, card4, card6은 범주형 변수가 확실함(결측치가 존재함)
- P_emaildomain, R_emaildomain은 구매자의 이메일 도메인으로 보이며, 두 feature는 서로 관계가 있을 것으로 보임
- card1, card2, card3, card5는 히스토그램 등으로 확인해본 결과, 연속형 변수가 확실함
- 많은 feature들에서 많은 결측치가 포함되어 있음 -> 결측치의 분포를 확인할 필요가 있음
- C3을 제외한 C1 ~ C14는 모두 연속형 변수인 것으로 보이며, 정보가 Masking되어있어 대략적인 의미라도 추측해야 함
- C3는 float형이지만 전체 Unique값 개수가 27개이므로 범주형으로 봐야할 것 같음
- M1 ~ M9는 매칭 정보이며, 전부 범주형 범주임 (T는 일치함, F는 일치하지 않는다는 의미로 보이며, 모두 결측치가 존재함)

## 2.4 범주형 변수(Categorical feature) 탐색
- 탐색하기 위해서 결측치를 임시로 문자로 변환함
- barplot을 사용하여 변수(feature)별 분포 확인함
- groupby를 사용하여 변수와 특징간의 관계를 확인함

### ProductCD feature

In [ ]:
#결측치 개수 확인
train_transaction.ProductCD.isnull().sum()

- 결측치가 없으므로, 문자로 변환할 필요가 없음

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))
# ProductCD의 거래 비율에 대한 bar plot 생성
train_transaction.ProductCD.value_counts(normalize=True).plot(kind='bar',ax = ax1)
# ProductCD별 사기 거래 비율에 대한 bar plot 생성
train_transaction.groupby('ProductCD').isFraud.mean().plot(kind='bar',ax=ax2)

In [ ]:
train_transaction.groupby('ProductCD').isFraud.mean()

- 상품별로 차이가 존재한다는 것을 확인함
    - W 상품이 0.7이상으로 가장 높은 거래 비율을 차지하고 S 상품이 1%에 가까운 가장 낮은 거래 비율을 보여줌
    - 사기거래 비율을 보면 두 번째로 높은 거래 비중을 차지하는 C 상품이 약 11.7%로 가장 높은 사기 거래 비율을 보여줌
    - W상품은 가장 높은 거래 비율을 차지하고 있지만 해당 상품의 사기거래 비율은 약 2% 5개 제품중 가장 적은 사기 거래 비율을 차지함
    - 전체 사기거래의 비율이 약 3.5%인데 C 상품과 H 상품, S 상품은 각각 11.7%와 4.8%, 5.9%로 전체 비율에 비해 높고 W 상품은 2%로 낮음
- ProductCD feature는 Unique값의 수가 적기 때문에 one-hot encoding이나 label encodeing을 하기로 결정함

### card4 feature

In [ ]:
#결측치 개수 확인
print(train_transaction.card4.isnull().sum()) # 결측치 1577개 
print(train_transaction.card4.isnull().sum() / train_transaction.shape[0]) #결측치 비율 0.3% 밖에 되지 않음

- 결측치를 전부 최빈값으로 대체해도 전체 비율에 크게 영향을 주지 않을 것으로 판단됨
- 이제 card 4의 결측치데이터가 사기거래 여부에 대해 유의미한지 판단해야함

In [ ]:
# 결측치에 해당하는 데이터의 사기러래 비율을 확인하고 만약 결측치 데이터가 사거거래 여부에 유의미한 비율을 차지한다면 
# 결측치를 다른 값으로 채우면 안 되며, 결측치 여부로 새로운 변수를 생성할 수 있음
# 하지만 결측치 데이터가 사기거래 여부에 별 영향이 없다면 결측치를 최빈값으로 대체해도 문제없다고 판단가능함
card4_df = train_transaction[['isFraud','card4']]
#결측치를 나타내는 feature 생성
card4_df['NA_card4'] = card4_df.card4.isnull().astype(int)
print(card4_df.groupby('NA_card4')['isFraud'].mean())

del card4_df
x= gc.collect()

- 결측치여부에 따른 사기거래 비율을 보게 되면 결측치가 아닐 때 사기거래 비율이 3.5% / 결측치일 때 사기거래 비율이 2.6%로 큰 비율 차이가 없다고 판단함
- 따라서 결측치를 최빈값으로 대체하기로 결정함

In [ ]:
#value_counts에 대한 barplot 생성
train_transaction.card4.value_counts(normalize = True).plot(kind = 'bar')

- visa와 mastercard가 매우 높은 비율(95%이상)을 차지하며, american express와 discover는 거의 없음

In [ ]:
#card4 feature에 대해 사기거래 비율을 계산
train_transaction.groupby('card4')['isFraud'].mean()

- 거래의 대다수를 차지하는 visa와 mastercard의 비율 값이 약 0.034로 유사함
- discover의 사기거래의 비율이 약 0.077로 전체의 약 8%이지만 discover의 거래횟수 자체가 워낙 작기 때문에 사기거래에 영향을 주는 요소라고 판단하기 어려움

### card6 feature

In [ ]:
#결측치 개수 확인
print(train_transaction.card6.isnull().sum()) # 결측치 1571개 
print(train_transaction.card6.isnull().sum() / train_transaction.shape[0]) #결측치 비율 0.3% 밖에 되지 않음

- 결측치를 전부 최빈값으로 대체해도 전체 비율에 크게 영향을 주지 않을 것으로 판단됨
- 이제 card 6의 결측치데이터가 사기거래 여부에 대해 유의미한지 판단해야함

In [ ]:
# 결측치에 해당하는 데이터의 사기러래 비율을 확인하고 만약 결측치 데이터가 사거거래 여부에 유의미한 비율을 차지한다면 
# 결측치를 다른 값으로 채우면 안 되며, 결측치 여부로 새로운 변수를 생성할 수 있음
# 하지만 결측치 데이터가 사기거래 여부에 별 영향이 없다면 결측치를 최빈값으로 대체해도 문제없다고 판단가능함
card6_df = train_transaction[['isFraud','card6']]
#결측치를 나타내는 feature 생성
card6_df['NA_card6'] = card6_df.card6.isnull().astype(int)
print(card6_df.groupby('NA_card6')['isFraud'].mean())

del card6_df
x= gc.collect()

- 결측치여부에 따른 사기거래 비율을 보게 되면 결측치가 아닐 때 사기거래 비율이 3.5% / 결측치일 때 사기거래 비율이 2.5%로 큰 비율 차이가 없다고 판단함
- 따라서 결측치를 최빈값으로 대체하기로 결정함

In [ ]:
#value_counts에 대한 barplot 생성
train_transaction.card6.value_counts(normalize = True).plot(kind = 'bar')

- debit와 credit이 대다수를 차지함
- debit or credit, charge card는 그래프에서 눈으로 확인이 불가능한 수준임
- 따라서 실제 수치값(비율)을 출력해봤음

In [ ]:
print(train_transaction.card6.value_counts(normalize = True))
print('\n')
print(train_transaction.card6.value_counts(normalize = False))

- 실제로 수치적으로 debit or credit, charge card는 각각 샘플의 수가 30개, 15개로 빈도가 매우 적고 전체 비율도 0%에 가까움

In [ ]:
#각 항목별 사기거래의 비율을 계산
train_transaction.groupby('card6')['isFraud'].mean()

- credit과 debit 사이에 차이가 존재함
- debit or credit과 charge card는 사기거래일 확률이 0%임
    - 하지만 전체 거래 빈도 자체가 너무 적어서 없는지, 실제로 없는 것인지 알 수가 없음(빈도가 적어서일 확률이 높음)
    - 따라서 0%라고 하여도 큰 의미가 없음
- card6 feature는 두 범주가 사기 거래인 데이터가 없으므로 사기거래 데이터가 있는 credit인지 아닌지(debit인지)여부를 나타내는 binary 변수로 변환

### P_emaildomain과 R_emaildomain feature
- 이전 탐색에서 두 feature 각각의 Unique값의 개수가 59, 60임을 확인함
- 또한, 결측치가 많았는데 email이 결측치라는 것은 이용자가 메일츨 기입하지 않았다는 의미로 해셕할 수 있음
- 메일을 적지 않았다는 의미에서 결측치와 사기거래여부와의 관계성을 살펴볼 필요가 있음
- feature의 이름에서 두 feature간의 관계가 있을 것이라고 추축이 가능함

In [ ]:
#결측치 개수 확인
print('P_emaildomain missing values')
P_missing_count = train_transaction.P_emaildomain.isnull().sum()
print(P_missing_count) # 결측치 94456개 
print(P_missing_count / train_transaction.shape[0]) #결측치 비율 약 16%
print('\n')
print('R_emaildomain missing values')
R_missing_count = train_transaction.R_emaildomain.isnull().sum()
print(R_missing_count) # 결측치 453249개 
print(R_missing_count / train_transaction.shape[0]) #결측치 비율 무려 약 77%

del P_missing_count, R_missing_count
x = gc.collect()

- 두 feature 모두 결측치의 비율이 상당히 높은데 특히 R_emaildomain의 결측치 비율이 77%나 되어 최빈값으로 채우는 것은 해당 feature의 분포에 영향을 줄 수 있을 것이라고 판단됨
- 결측치데이터가 사기거래 여부에 대해 어떤 유의미한 점이 있는지 판단해야함

In [ ]:
emaildomain_df = train_transaction[['isFraud','P_emaildomain','R_emaildomain']]

#결측치를 나타내는 feature 생성
emaildomain_df['NA_P_emaildomain'] = emaildomain_df.P_emaildomain.isnull().astype(int)
emaildomain_df['NA_R_emaildomain'] = emaildomain_df.R_emaildomain.isnull().astype(int)

#결측치 여부에 따른 사기거래 비율
print(emaildomain_df.groupby('NA_P_emaildomain')['isFraud'].mean())

- P_emaildomain는 결측치가 아닌 경우, 사기거래의 비율이 약 3.6%이고 결측치인 경우, 사기거래의 비율이 약 3%정도이기 때문에 크게 문제 없어 보임

In [ ]:
print(emaildomain_df.groupby('NA_R_emaildomain')['isFraud'].mean())

del emaildomain_df
x= gc.collect()

- R_emaildomain은 오히려 결측치가 아닌 경우에 사기거래의 비율이 약 8.2%이고, 결측치의 경우, 사기거래의 비율이 약 2.1%로 나타남
- **결측치인 경우보다 결측치가 아닌 경우의 사기거래가 4배 이상 많음**
- 따라서, 해당 정보를 변수로 활용할 예정임

In [ ]:
# train과 test 변수에 R_emaildomain의 결측치 여부 binary 정보를 변수로 추가함
train_transaction['NA_R_emaildomain'] = train_transaction.P_emaildomain.isnull().astype(int)
test_transaction['NA_R_emaildomain'] = test_transaction.P_emaildomain.isnull().astype(int)

In [ ]:
#각 Unique값(email domain)별로 데이터 수가 몇개인지 확인
train_transaction.P_emaildomain.value_counts(normalize = False)

- domain 값이 제대로 정리되지 않고 'gmail.com' / 'gmail'과 같은 형태로 '.com'이 붙지 않은 형태도 존재함
- 그리고 domain은 같은데 '.com, .co.uk, .co.jp와 같이 서로 다르게 표현된 email도 존재함
    - 따라서, '.'을 기준으로 domain 주소값과 com, net과 같은 domain들을 분리하여 domain 주소값만 가져와서 봐야할 필요가 있음 

In [ ]:
#domain 주소값만 가져와서 추가적인 탐색 수행
train_transaction['P_emaildomain'] = train_transaction['P_emaildomain'].str.split('.').str[0]
train_transaction['R_emaildomain'] = train_transaction['R_emaildomain'].str.split('.').str[0]

test_transaction['P_emaildomain'] = train_transaction['P_emaildomain'].str.split('.').str[0]
test_transaction['R_emaildomain'] = train_transaction['R_emaildomain'].str.split('.').str[0]

In [ ]:
train_transaction.P_emaildomain.value_counts(normalize = False)

In [ ]:
#각 domain 주소별 사기거래 비율 확인
train_transaction.groupby('P_emaildomain')['isFraud'].mean().sort_values(ascending=False)

- P_emaildomain에 따른 사기거래 비율의 차이가 존재함
- 다만, 사기거래 비율이 약 40%로 가장 높은 protonmail의 경우 샘플 수가 76개이고, 약 19%인 mail은 559개, aim은 315개임
- 전체 데이터 수가 54만개 정도인 것을 생각하며 그 수가 매우 적어서 유의한 것인지 판단하기 애매함
- 만약 변수로 사용한다면 사기거래 비율이 반올림하여 약 10%이상 되는 protonmail, mail, aim, outlook을 그룹으로 묶어 해당 그룹인지 아닌지 여부로 사용할 것임

In [ ]:
train_transaction.R_emaildomain.value_counts(normalize = False)

In [ ]:
train_transaction.groupby('R_emaildomain')['isFraud'].mean().sort_values(ascending=False)

- P_emaildomain과 마찬가지로, R_emaildomain에 따른 사기거래 비율의 차이가 존재함
- 다만, P_emaildomain과 같이 사기거래 비율이 약 95%로 가장 높은 protonmail의 경우 샘플 수가 41개이고, 약 38%인 mail은 122개임
- 전체 데이터 수가 54만개 정도인 것을 생각하며 그 수가 매우 적어서 유의한 것인지 판단하기 애매함
- 만약 변수로 사용한다면 사기거래 비율이 반올림하여 약 10%이상 되는 protonmail, mail, outlook, icloud, gmail을 그룹으로 묶어 해당 그룹인지 아닌지 여부로 사용할 것임
- P_emaildomain의 경우와 R_emaildomain의 경우를 비교해보면, 사기거래의 비율이 높은 도메인 주소가 몇개 겹침(protonmail, mail, outlook)
- 따라서, P_emaildomain와 R_emaildomain가 같은 거래인지 여부를 확인함

In [ ]:
# 각 거래에 대해(같은 거래에 대해) P_emaildomain와 R_emaildomain의 값이 서로 같은지 여부를 확인함
print((train_transaction.P_emaildomain == train_transaction.R_emaildomain).astype(int).value_counts(False))
print('\n')
print((train_transaction.P_emaildomain == train_transaction.R_emaildomain).astype(int).value_counts(True))
print('\n')

# P_emaildomain와 R_emaildomain의 값이 서로 같은지 여부에 따른 사기거래의 비율 확인
emaildomain_df = train_transaction[['P_emaildomain','R_emaildomain','isFraud']]
emaildomain_df['same_emaildomain'] = (emaildomain_df.P_emaildomain == emaildomain_df.R_emaildomain).astype(int)
print(emaildomain_df.groupby('same_emaildomain')['isFraud'].mean())

del emaildomain_df
x = gc.collect()

- 같은 doamin 주소인 거래의 데이터 샘플 수는 총 102523개이며 전체 대비 약 17%에 해당함 -> 데이터 샘플 수도 굉장히 많은 편임
- domain의 주소가 같은 거래의 경우 사기인 비율이 약 9.6%로 거의 10%에 가까우며, 주소가 다른 경우에는 사기 비율이 약 2.2%로 매우 낮음
- 실제 사기 거래의 비율이 3.5%인데 같은 domain의 거래의 약 10%가 사기임
- domain 주소가 같은 거래의 수가 전체의 17%로 굉장히 많은데 이에 따른 사기 비율이 약 4배이상 차이가 나기 때문에 이는 매우 유의미하다고 판단할 수 있음
- 따라서, 해당 정보를 변수로 활용할 예정임

In [ ]:
#train과 test 데이터에 same_emaildomain이라는 두 feature가 서로 같은지 여부에 대한 binary 값을 설명변수로 추가함
train_transaction['same_emaildomain'] = (train_transaction.P_emaildomain == train_transaction.R_emaildomain).astype(int)
test_transaction['same_emaildomain'] = (test_transaction.P_emaildomain == test_transaction.R_emaildomain).astype(int)

### C3 feature
- 결측치가 없다는 것을 이전에 확인하였음
- 데이터 type은 float이지만, Unique값의 개수가 총 27개로 많지 않음 -> 일단은 범주형으로 판단함

In [ ]:
train_transaction.C3.value_counts(normalize=False)

In [ ]:
train_transaction.C3.value_counts(normalize=True)

- 거의 모든(약 99.6%) 데이터가 0의 값임을 알 수 있음

In [ ]:
train_transaction.groupby('C3')['isFraud'].mean()

- 2 이상의 값은 전체 대비 샘플 수가 너무 적기 때문에 사기비율이 0으로 나왔을 것으로 추측함
- 0(약3.5%)은 전체의 99.6%이기 때문에 전체 데이터의 사기거래비율(약 3.5%)과 유사함
- 1은 샘플 수가 2137로 어느정도 있음에도 사기거래 비율이 약 0.2%로 전체 사기비율과 크게 차이남
- C3값이 0인지 0이 아닌지에 따른 차이가 존재할 것이라는 추측이 가능함

### M 관련 변수(features)
- 모두 범주형 변수이며, 이전에 확인한 것처럼 값이 대부분 T/F의 값을 가지고 있기 때문에 같이 탐색함

In [ ]:
#M column 이름 모음
M_cols = ['M'+str(i) for i in range(1,10)]
for M_col in M_cols:
    print(f'{M_col} missing values')
    #결측치 개수 확인
    count = train_transaction[M_col].isnull().sum()
    print('count:', count)
    #결측치 비율 확인
    print('ratio: %.2f'%(count / train_transaction.shape[0]))
    print('======================')
    print(f'{M_col} values')
    #변수 분포 확인
    print( train_transaction[M_col].value_counts())
    print('\n')

- 모든 feature들에 대한 결측치의 비율이 약 50% 가까이 또는 그 이상으로 매우 많으므로 단순히 다른 값으로 대체하거나 제거하지 못함
- M4 feature를 제외하고 나머지 모든 feature들은 T/F의 값을 가짐

In [ ]:
#탐색을 위해, 모든 결측값을 잠시 대체함
M_df = train_transaction[M_cols+['isFraud']]
M_df[M_cols] = M_df[M_cols].fillna('MV') #missing vlaue

for col in M_cols:
    print(M_df.groupby(col)['isFraud'].mean())
    print('\n')
    
del M_cols, count, M_df
x = gc.collect()

- M4,M5를 제외한 모든 feature들에서 결측치를 의미하는 'MV'일 경우 사기거래 비율이 다른 T/F보다 높음
- M4의 경우 결측치일 때 오히려 사기비율이 약 2%로 다른 결측치가 아닐 때 다른 class보다 낮음
- 하지만 확실히 대부분의 feature들에서 결측치를 의미할 경우와 결측치가 아닌 경우의 사기거래 비율에서 확실한 차이를 보이고 있으므로 결측치에 대해서 하나의 범주로 판단하기로 결정함

### DeviceType feature

In [ ]:
#결측치 개수 확인
print(train_identity.DeviceType.isnull().sum()) # 결측치 3423개 
print(train_identity.DeviceType.isnull().sum() / train_identity.shape[0]) #결측치 비율 약 2.4%임


In [ ]:
DeviceType_df = train_identity[['TransactionID','DeviceType']].merge(train_transaction[['TransactionID','isFraud']], how = 'right', on='TransactionID')
#train_transaction 데이터와 합친 데이테에 대한 결측치 개수 확인
print(DeviceType_df.DeviceType.isnull().sum()) # 결측치 449730개 
print(DeviceType_df.DeviceType.isnull().sum() / DeviceType_df.shape[0]) #결측치 비율 약 76%임
print('\n')
#결측치를 나타내는 feature 생성
DeviceType_df['NA_DeviceType'] = DeviceType_df.DeviceType.isnull().astype(int)
print(DeviceType_df.groupby('NA_DeviceType')['isFraud'].mean())

del DeviceType_df
x= gc.collect()

- identity에서는 결측치가 3423개, 전체 약 2.4%였지만, transaction 데이터를 합치니까 결측치가 449730개에 76%로 증가함
    - 결측치에 해당되는 전체의 2.4%해당 되는 소수의 ID들의 거래 횟수가 전체 거래횟수의 약 76%를 차지한다는 의미임
- 앞서 살펴본 R_emaildomain와 유사하게 오히려 결측치가 아닌 경우에 사기거래의 비율이 약 8%이고, 결측치의 경우, 사기거래의 비율이 약 2.1%로 나타남
- **결측치가 아닌 경우의 사기거래가 결측치인 경우보다 약 4배 가까이 많음**
- 따라서, 해당 정보를 변수로 활용할 예정임

## 2.5 연속형 변수(Numerical feature) 탐색
- 변수별 분포 확인(histogram)
- 변수와 Target간 관계 파악 (boxplot)

# 3. Data Preprocessing
- Categorical Features Preprocessing (Encoding)
- Feature Selection
- Feature Engineering
- PDA 

## Merge Transaction data and Identity data
- train, test에 대해 transaction과 identitiy 데이터를 합쳐서 사용함

In [ ]:
# 처음에 미리 데이터의 index를 TransactionID로 설정한 경우
if is_index_TransactionID:
    X_train = train_transaction.merge(train_identity, how='left', left_index=True, right_index=True)
    X_test = test_transaction.merge(test_identity, how='left', left_index=True, right_index=True)
else:
# 처음에 미리 데이터의 index를 TransactionID로 설정하지 않은 경우
    # 'TransactionID' feature를 기준으로 transaction, identity 데이터 합치기
    X_train = pd.merge(train_transaction, train_identity, how = 'left', on = 'TransactionID')
    X_test = pd.merge(test_transaction, test_identity, how = 'left', on = 'TransactionID')
    # 'TransactionID' feature를 index로 설정함
    X_train = X_train.set_index('TransactionID')
    X_test = X_test.set_index('TransactionID')

del train_identity, test_identity, train_transaction, test_transaction
tmp = gc.collect()

## 설명변수(feature)와 반응변수(target)를 분리

In [ ]:
# target_col = 'isFraud'
# feature_col = X_train.columns.difference([target_col])
# y_train = X_train[target_col].copy()
# X_train = X_train[feature_col]

y_train = X_train.pop['isFraud']
#X_train.drop(['isFraud'], axis = 1, inplace = True)

## Normalize D Columns (D feature 정규화)

- D feature는 과거의 어떤 순간부터의 거래 시점까지의 "Time delta"값임
- D feature를 델타값이 아닌 그 과거의 시점의 값으로 변환하여 해당 feature가 시간에 따라 증가하는 특성을 없애
  시계열의 특성을 제거한 모델에 조금 더 의미있는 feature로 사용함
- 단, D1의 카드만든 이후 지금까지의 기간과 같이 델타값이 의미가 있는 feature는 제외함

In [ ]:
# Normalize D features
for idx in range(1,16):
    # 일부 feature는 제외함
    if idx in [1,2,3,5,9]: 
        continue
    # 델타값(일단위값) - (거래시점(초단위)/24*60*60 -> 초단위값을 일단위로 변환)
    X_train['D'+str(idx)] =  X_train['D'+str(idx)] - X_train.TransactionDT/np.float32(24*60*60)
    X_test['D'+str(idx)] = X_test['D'+str(idx)] - X_test.TransactionDT/np.float32(24*60*60) 

- test의 id feature를 train의 id feature와 이름을 통일시킴

In [ ]:
for column in X_test.columns:
    if column.startswith('id'):
            X_test.rename(columns={column:column.replace('-','_')},inplace=True)

## Feature Selection

- 결측치가 N개 이상인 feature들을 모두 제거함으로써 feature를 골라냄

In [ ]:
def drop_N_missing_values_columns(df_train, df_test,N=100000):
    
    def getNulls(data):
        #결측치 개수 및 비율 계산
        total = data.isnull().sum()
        percent = data.isnull().sum() / data.isnull().count()
        missing_data = pd.concat([total, percent], axis = 1, keys = ['total', 'precent'])

        return missing_data

    # Train 데이터의 결측치를 파악함
    missing_data_train = getNulls(df_train)

    # 결측치가 N개 이상있는 경우, 해당 feature 버림
    sel_cols = missing_data_train[missing_data_train['total'] > N].index
    del missing_data_train

    # Drop the columns
    df_train.drop(sel_cols, axis = 1, inplace = True)
    df_test.drop(sel_cols, axis = 1, inplace = True)

In [ ]:
#N = 100000
#N defalt 값으로 100000으로 설정 
drop_N_missing_values_columns(X_train, X_test)

## Handle Missing Values

- 결측치를 최빈값이나 평균값으로 채우는지, -1로 채우는지에 따라 방법이 다름

In [ ]:
def handle_missing_values_mean_mode(df_train, df_test):
    ntrain = df_train.shape[0]
    ntest = df_test.shape[0]
    #train , test데이터를 합침
    df_all = pd.concat([df_train, df_test], axis = 0, sort = False)
    #모든 데이터에 대한 column명을 가져옴
    all_data_cols = df_all.columns

    # 최빈값으로 결측치 채움
    for i in all_data_cols:
        # str값의 경우 최빈값으로 결측치를 채움
        if df_all[i].dtype == 'object':
            df_all[i] = df_all[i].fillna(df_all[i].mode()[0])
        # C 또는 V feature의 경우에 최빈값으로 결측치를 채움
        elif (i.startswith("C") or (i.startswith("V"))) and df_all[i].isnull().sum() > 0:
            df_all[i] = df_all[i].fillna(df_all[i].mode()[0])

    # 평균값으로 결측치 채움
    df_all['card2'] = df_all['card2'].fillna(df_all['card2'].mean())
    df_all['card3'] = df_all['card3'].fillna(df_all['card3'].mean())
    df_all['card5'] = df_all['card5'].fillna(df_all['card5'].mean())
    df_all['D1'] = df_all['D1'].fillna(df_all['D1'].mode()[0])
    df_all['D10'] = df_all['D10'].fillna(df_all['D10'].mode()[0])
    df_all['D15'] = df_all['D15'].fillna(df_all['D15'].mode()[0])
    df_all['addr1'] = df_all['addr1'].fillna(df_all['addr1'].mean())
    df_all['addr2'] = df_all['addr2'].fillna(df_all['addr2'].mean())

    # 다시 train과 test 데이터로 나눔
    df_train = df_all[:ntrain]
    df_test = df_all[ntrain:]
    
    del df_all 
    gc.collect()

def handle_missing_values_negative_one(df_train, df_test):
    for col in df_train.columns:
        #숫자값 데이터를 가지는 feature에 대해
        if not df_train[col].dtype=='object':
            #'TransactionAmt','TransactionDT' feature를 제외한 feature들에 대해
            if col not in ['TransactionAmt','TransactionDT']:
                #각 feature의 최소값을 구해 
                mn = np.min((df_train[col].min(),df_test[col].min()))
                #각 feature의 모든 값들에 대해 최소값을 빼주어 양수로 만듦
                df_train[col] -= np.float32(mn)
                df_test[col] -= np.float32(mn)
                #결측치를 모두 -1로 설정함
                df_train[col].fillna(-1,inplace=True)
                df_test[col].fillna(-1,inplace=True)

In [ ]:
 handle_missing_values_negative_one(X_train, X_test)

### Encode the categorical features (Label Encode)

In [ ]:
#1) 원핫인코딩(one-hot-encoding)
def encode_one_hot (df_train, df_test):
    df_train = pd.get_dummies(df_train)
    df_test = pd.get_dummies(df_test)

#2) factorize함수로 직접 Label encode 하기
def encode_label (df_train, df_test):
    # 데이터를 라벨숫자로 변환함
    for col in df_train.columns:
        if df_train[col].dtype=='object': 
            df_comb = pd.concat([df_train[col],df_test[col]],axis=0)
            df_comb,_ = df_comb.factorize(sort=True)
            X_train[col] = df_comb[:len(df_train)].astype('int16')
            X_test[col] = df_comb[len(df_train):].astype('int16')
            
#3) Label Encoder() 사용하기
from sklearn.preprocessing import LabelEncoder
def label_encoder (df_train, df_test):
    for col in df_train.columns:
        


In [ ]:
#one-hot-encoding/ labelencoderlibrary/ label-encoding
encode_label(X_train, X_test)


# 4. Machine Learning Modeling 

## Validation for Machine Learning Model
- 시계열 데이터는 Random Sampling을 하면 안됨
- Train엔 과거 데이터, Test엔 (과거 대비) 미래 데이터가 있어야 함
- 해당 데이터가 시계열 데이터이긴 하지만 예측하고자 하는 Target이 시간에 영향을 받을 수 있는 수요값같은 게 아니라 사기거래 여부임
- 각각의 거래에 대한 정보들을 통해 사기거래 여부를 판별하는 binary classification 문제이므로 데이터를 섞어도 상관없음

### 1) Split of Train dataset and Validation dataset by using Stratified Random Sampling
- Train data만으로 성능을 평가해보고 데이터에 대한 모델의 성능을 개선하는 데에 있어서 하이퍼 파라미터를 조절하거나 타겟 데이터에 대한 Oversampling을 적용하거나 Bagging을 적용한 결과에 대한 성능평가를 해보기 위해 Train dataset을 Train data(훈련 데이터)와 Validation data(검증 데이터)로 나눔
- 해당 문제의 타겟 데이터의 class 비율이 imbalance하므로 Scikit-learn에 있는 model_selection의 train_test_split()에서 stratify파라미터와 StratifiedShuffleSplit()을 사용하여 y_train의 class비율에 따라 데이터를 나누는 층화추출(Stratified Sampling)기법을 사용함
- train_test_split()함수의 stratify파라미터를 y_train으로 설정하여 y_train의 class비율에 따라 데이터를 나눔
- StratifiedShuffleSplit()함수에 X_train, y_train을 파라미터로 넣어 X_train에 대해서 y_train의 class 비율에 따라 층화추출함

In [ ]:
#방법(1) sklearn.model_selection.train_test_split 함수를 이용한 Train, Test set 분할 
#(층을 고려한 X_train, X_test, y_train, y_test 반환)
def split_validation_dataset(X_train=X_train, y_train=y_train, val_size=0.2, shuffle=True, stratify=y_train):
    from sklearn.model_selection import train_test_split
    
    return train_test_split(X_train, y_train, stratify=stratify, test_size=val_size, shuffle=shuffle)

#방법(2) sklearn.model_selection.StratifiedShuffleSplit 함수를 이용한 Train, Test set 분할
#(층을 고려한 train/test index 반환 --> Train, Test set indexing)
def split_validation_index(X_train=X_train, y_train=y_train, val_size=0.2):
    from sklearn.model_selection import StratifiedShuffleSplit
    #1개의 train/ test set 만을 분할하므로 n_splits=1 로 지정해주며, test_size에 test set의 비율을 지정해주고, 
    #random_state에는 재현가능성을 위해 난수 초기값으로 아무값이 지정해줍니다.
    split_object = StratifiedShuffleSplit(n_splits=1, test_size=0.2)
    return split_object.split(X_train, y_train)

In [ ]:
# TRAIN 80% Validation 20%

#CASE1: Split DataFrame of Train data
X_train, X_val, y_train, y_val = split_validation_dataset()

#CASE2: Split Index of Train data
# idxT, idxV = split_validation_index()
# X_train = X_train[idxT]
# X_val = X_train[idxV]
# y_train = y_train[idxT]
# y_test = y_train[idxV]

# TRAIN 75% Validation 25%
# idxT = X_train.index[:3*len(X_train)//4]
# idxV = X_train.index[3*len(X_train)//4:]


print(X_train.shape)
print(X_val.shape)
print(y_train.shape)
print(y_val.shape)

#### 나누어진 Dataset의 Target 데이터의 class비율대로 나누어졌는지 데이터 분포 확인

In [ ]:
# 나뉘어진 반응 변수의 분포 확인(Class imbalance 확인)
print('Number of y_train data is', len(y_train))
print('Number of y_val data is', len(y_val))
print('\n')
from collections import Counter
target_train_dist = Counter(y_train)
target_val_dist = Counter(y_val)
print(target_train_dist)
print(target_val_dist)
print('\n')
print('Ratio of fraud transaction data by train data is %.1f%%'%(target_train_dist[0] / len(y_train)*100))
print('Ratio of non-fraud transaction data by train data is %.1f%%'%(target_train_dist[1]/ len(y_train)*100))
print('\n')
print('Ratio of fraud transaction data by validation data is %.1f%%'%(target_val_dist[0] / len(y_val)*100))
print('Ratio of non-fraud transaction data by validation data is %.1f%%'%(target_val_dist[1]/ len(y_val)*100))

### 2) Cross Validation(CV)
- 교차검증을 수행함
- class(Label)가 imbalance하므로 Stratified K-Fold CV을 적용함
- 따라서 Scikit-learn의 model_selection에 있는 StratifiedKFlod()함수 사용함

In [ ]:
from sklearn.model_selection import StratifiedKFold



## Oversampling of Target data

In [ ]:
def oversampling_target_data(X_train=X_train, y_train=y_train, ratio=0.3, method='SMOTE'):
    if method == 'SMOTE':
        from imblearn.over_sampling import SMOTE

        smote = SMOTE(ratio = 0.3) # SMOTE 알고리즘, 비율 증가
        X_train_res, y_train_res = smote.fit_sample(X_train, y_train.ravel()) # Over Sampling 진행
        
    elif method =='BorderlineSMOTE':
        from imblearn.over_sampling import BorderlineSMOTE
    
    elif method =='ADASYN':
        from imblearn.over_sampling import ADASYN
    

## Modeling Method

In [ ]:
import xgboost as xgb
xgmodel = xgb.XGBClassifier( 
        n_estimators=2000,
        max_depth=12, 
        learning_rate=0.02, 
        subsample=0.8,
        colsample_bytree=0.4, 
        missing=-1, 
        eval_metric='auc',
        # USE CPU
        #nthread=4,
        #tree_method='hist' 
        # USE GPU
        tree_method='gpu_hist' 
    )
# xgmodel = xgb.XGBClassifier(n_estimators = 5000,
#                             #max_depth = 12,
#                             #learning_rate = 0.02,
#                             #subsample = 0.8,
#                             #colsample_bytree = 0.4,
#                             #missing = -1,
#                             #random_state = 42,
#                             #tree_method = 'gpu_hist')
xgmodel.fit(X_train.loc[idxT,:], y_train[idxT],eval_set=[(X_train.loc[idxV,:],y_train[idxV])],
        verbose=50, early_stopping_rounds=100)

LightGBM

## Ensemble of Ensemble Method